In [1]:
from utils import *
from massfunction import *

import numpy as np
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import os
import emcee
import sys
import numpy as np
import pickle

In [2]:
cosmos_f = open('data/cosmo_params.pkl', 'rb')
cosmo_params = pickle.load(cosmos_f) #cosmo_params is a dict
cosmos_f.close()

In [11]:
X = []
Y = []
for box in cosmo_params:
    X += [box]
    Y += [[cosmo_params[box][key] for key in cosmo_params[box]]]
    
X = np.array(X)
Y = np.array(Y)

from sklearn.preprocessing import StandardScaler

# Assuming Y is your input matrix with shape (n_samples, n_features)
# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(Y)

# Standardize the data
Y = scaler.transform(Y)

In [15]:
for i in range(len(Y[0])):
    print(np.mean(Y[:,i]), np.std(Y[:,i]))

-9.947598300641404e-16 1.0000000000000002
2.05702122002549e-14 0.9999999999999996
2.250051996573651e-16 1.0000000000000002
-1.1783167034688328e-15 0.9999999999999999
1.3174646558885192e-16 1.0000000000000004
1.5543122344752192e-15 0.9999999999999998
1.9539925233402755e-16 1.0
-4.263256414560601e-16 1.0


In [16]:
#TODO remove boxes thta crash when prepare data

In [17]:
#draw edge between boxes that are nearest neighbors 
from scipy.spatial import KDTree

In [18]:
kdtree = KDTree(Y)

In [79]:
edges = {box:[] for box in X}
for i in range(len(Y)):
    cbox = X[i]
    curr_loc = Y[i]
    
    d, i = kdtree.query(curr_loc, k=10)
    
    #get rid of self
    d = d[1:]
    i = i[1:]
    
    min_d = d[0]
    for (dc, ic) in zip(d,i):
        if(np.abs(dc-min_d)/min_d < 0.1):
            edges[cbox] += [X[ic]]
            edges[X[ic]] += [cbox]
        else:
            break

In [80]:
from collections import deque
visited = set()

def bfs(graph, start_node):
    queue = deque([start_node])
    
    while queue:
        node = queue.popleft()
        if node not in visited:
            print(node)  # or perform any other operation on the node
            
            visited.add(node)
            neighbors = graph[node]
            
            for neighbor in neighbors:
                if neighbor not in visited:
                    queue.append(neighbor)

def bfs_disconnected(graph):
    n_clusters = 0
    for node in graph:
        if node not in visited:
            print('STARTING CLUSTER')
            n_clusters += 1
            bfs(graph, node)
            print('ENDING CLUSTER')
            print()
    return n_clusters
n_clusters = bfs_disconnected(edges)

STARTING CLUSTER
Box0_1400
Box_n50_11_1400
Box_n50_12_1400
Box15_1400
ENDING CLUSTER

STARTING CLUSTER
Box1_1400
Box26_1400
Box51_1400
Box22_1400
Box_n50_40_1400
Box_n50_41_1400
Box4_1400
Box54_1400
Box34_1400
Box_n50_27_1400
Box49_1400
Box68_1400
Box_n50_48_1400
Box_n50_10_1400
Box_n50_30_1400
Box_n50_20_1400
Box45_1400
Box92_1400
Box_n50_14_1400
Box_n50_13_1400
Box_n50_6_1400
Box83_1400
Box47_1400
Box59_1400
Box37_1400
Box_n50_25_1400
Box_n50_47_1400
Box_n50_15_1400
Box30_1400
Box82_1400
Box_n50_45_1400
Box_n50_38_1400
Box_n50_0_1400
Box_n50_24_1400
Box27_1400
Box72_1400
Box6_1400
Box_n50_43_1400
Box_n50_39_1400
Box52_1400
Box39_1400
Box14_1400
Box_n50_5_1400
Box99_1400
Box_n50_19_1400
Box_n50_34_1400
Box_n50_33_1400
Box_n50_35_1400
Box_n50_4_1400
Box94_1400
Box_n50_29_1400
Box88_1400
Box_n50_16_1400
Box96_1400
Box_n50_26_1400
Box77_1400
Box2_1400
Box91_1400
Box_n50_36_1400
Box_n50_32_1400
Box_n50_44_1400
Box28_1400
Box29_1400
Box12_1400
Box74_1400
Box8_1400
Box60_1400
Box20_1400
Box

In [81]:
print(n_clusters)

10
